# Text Embedding Basics

<div style="background-color: #d9f2f7ff; border:1px solid #a0a0a060; padding: 1rem; color: #151515">
✏️ <strong>Note:</strong> The examples below can be run in any order. Input and output will be automatically saved to the "output" folder.
</div>

You can create embeddings synchronously using the Amazon Bedrock Runtime `InvokeModel` function. Try the simple examples below.

In [ ]:
# Restore variables from setup notebook
%store -r s3_bucket
print(f"Using S3 bucket: {s3_bucket}")
%store -r region_name
print(f"Using region: {region_name}")

### Synchronous Embedding - Inline Text

This first example includes the source text for the embedding directly within the request using the `text.value` request parameter.

In [ ]:
from utils.utils import pretty_format
import nova_embeddings

request_body = {
    "taskType": "SINGLE_EMBEDDING",
    "singleEmbeddingParams": {
        "embeddingPurpose": "GENERIC_INDEX",
        "embeddingDimension": 256,
        "text": {"truncationMode": "END", "value": "Hello, world!"},
    },
}

body, metadata = nova_embeddings.generate_embedding_sync(request_body)

print("Request ID:", metadata.get("RequestId"))
print(pretty_format(body))

### Synchronous Embedding - S3 Text File

This example shows how to use a text file stored in S3 as the input for a text embedding. **Be sure to edit the `S3_TEXT_FILE` value before running this cell.**

In [ ]:
from utils.utils import pretty_format
import nova_embeddings

# Update this constant to reference a text file in your S3 bucket.
S3_TEXT_FILE = f"s3://{s3_bucket}/sample_assets/text/Flatland-Edwin_Abbott-excerpt.txt"

request_body = {
    "taskType": "SINGLE_EMBEDDING",
    "singleEmbeddingParams": {
        "embeddingPurpose": "GENERIC_INDEX",
        "embeddingDimension": 256,
        "text": {
            "truncationMode": "END",
            "source": {"s3Location": {"uri": S3_TEXT_FILE}},
        },
    },
}

body, metadata = nova_embeddings.generate_embedding_sync(request_body)

print("Request ID:", metadata.get("RequestId"))
print(pretty_format(body))

### Asynchronous Embedding - Inline Text

Use the **StartAsyncInvoke** function to generate a text embedding asynchronously. The Nova Embeddings asynchronous generation features is particularly useful for its ability to segment long input into multiple embeddings. The results will be stored to an S3 bucket you specify. Calls to StartAsyncInvoke will return an `invocationArn` that can be used to check the status of the async job using the **GetAsyncInvoke** function.

In the example below, we load a .txt file representing a book and send that text content to Nova Embeddings with a `maxLengthChars` setting of 1000 which will automatically segment the text into chunks of 1000 characters or fewer. This segmentation process attempts to segment at word boundaries, so each chunk may not be exactly 1000 characters.

Edit the `S3_DESTINATION_BUCKET` value below to point to your own S3 bucket. Then run the cell.

In [ ]:
import nova_embeddings

# We'll load a long text value from disk. ("Flatland" is a public domain book by Edwin Abbott.)
TEXT_FILE = "sample_assets/text/Flatland-Edwin_Abbott.txt"

with open(TEXT_FILE, "r", encoding="utf-8") as f:
    text_content = f.read()

request_body = {
    "taskType": "SEGMENTED_EMBEDDING",
    "segmentedEmbeddingParams": {
        "embeddingPurpose": "GENERIC_INDEX",
        "embeddingDimension": 256,
        "text": {
            "truncationMode": "END",
            "value": text_content,
            "segmentationConfig": {
                "maxLengthChars": 1000
            }
        },
    },
}

invocation_arn, metadata = nova_embeddings.generate_embedding_async(
    request_body, s3_bucket
)

print("Request ID:", metadata.get("RequestId"))
print("Invocation ARN:", invocation_arn)

You can use the **GetAsyncInvoke** function to check the status of your asynchronous embedding job.

In [ ]:
from utils.utils import pretty_format
import boto3

# Uncomment the line below if you'd like to use a hard-coded invocation_arn.
# invocation_arn = "<your-invocation-arn>"

if "invocation_arn" not in locals():
    raise SystemExit(
        "invocation_arn variable not found. Please run the previous cell first or hard code an invocation_arn in this cell."
    )

# Create the Bedrock Runtime client.
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name=region_name,
)

try:
    # Get the async job status.
    response = bedrock_runtime.get_async_invoke(
        invocationArn=invocation_arn,
    )

except Exception as e:
    # You would typically add your own exception handling here.
    print(e)

print(pretty_format(response))

The convenience function below will download the output for recent async jobs to the "output" directory. It will optionally monitor any in-progress jobs, downloading their output when they complete.

In [ ]:
from utils.async_job_utils import download_recent_async_jobs

# Download and monitor async jobs from the past N hours.
download_recent_async_jobs(output_dir="output", hours=4, wait_for_running_jobs=True)